# Implementation Based On Selenium Lib

In [ ]:
info_list = []

In [ ]:
# %%file SeleniumLiepin.py
import time
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

# Initialize variable 
page_number = 3
info_df = pd.DataFrame()

# Set up Edge driver
opt = webdriver.EdgeOptions()   # Create setup object
opt.add_argument('--disable-gpu')
driver = webdriver.Edge(options=opt)

# Wait for element locating
driver.implicitly_wait(10)

# Open the URL
try:
    # url = "https://www.liepin.com"
    url = 'https://www.liepin.com/zhaopin/?city=410&dq=410&pubTime=&currentPage=3&pageSize=40&key=%E8%87%AA%E7%84%B6%E8%AF%AD%E8%A8%80%E5%A4%84%E7%90%86%E5%B7%A5%E7%A8%8B%E5%B8%88&suggestTag=&workYearCode=0&compId=&compName=&compTag=&industry=&salary=&jobKind=&compScale=&compKind=&compStage=&eduLevel=&otherCity=&sfrom=search_job_pc&ckId=01q2gz6bfx7lccu04cvnflv403m991qb&scene=page&skId=01q2gz6bfx7lccu04cvnflv403m991qb&fkId=01q2gz6bfx7lccu04cvnflv403m991qb&suggestId='
    driver.get(url)

    # # Type and search job position
    # # Locate the `input` element and entry key word
    # driver.find_element(By.XPATH, "//input[@placeholder='搜索职位/公司/内容关键词']").send_keys("自然语言处理工程师")
    # # Locate the search button element and click the button
    # driver.find_element(By.XPATH, "//span[text()='搜索']").click()

    # Switch to recruitment page
    driver.switch_to.window(driver.window_handles[-1])

    while True:
        tmp_data_list = []
        tmp_data_df = pd.DataFrame()

        # Wait for page loading to be completed
        # while not driver.execute_script("document.readyState == 'complete'"):
        #     time.sleep(1)
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//div[@id='lp-search-job-box']/div[3]/section[1]/div[1]/div[40]")))
        eles = driver.find_elements(By.XPATH, "//div[@id='lp-search-job-box']/div[3]/section[1]/div[1]/div")

        # Extracte job information
        for ele in eles:
            tmp_data_dict = dict()
            try:
                tmp_data_dict['position'] = ele.find_element(By.XPATH, "./div/div[1]/div/a/div[1]/div/div[1]").text
            except NoSuchElementException:
                tmp_data_dict['position'] = 'N/A'  
            try:
                tmp_data_dict['city'] = ele.find_element(By.XPATH, "./div/div[1]/div/a/div[1]/div/div[2]/span[2]").text  
            except NoSuchElementException:
                tmp_data_dict['city'] = 'N/A'  
            try:
                tmp_data_dict['payment'] = ele.find_element(By.XPATH, "./div/div[1]/div/a/div[1]/span[@class='jsx-2693574896 job-salary']").text
            except NoSuchElementException:
                tmp_data_dict['payment'] = 'N/A'  
            try:
                tmp_data_dict['enterprise'] = ele.find_element(By.XPATH, "./div/div[1]/div/div/div/span").text
            except NoSuchElementException:
                tmp_data_dict['enterprise'] = 'N/A'  
            try:
                tmp_data_dict['requirement-list'] = ele.find_element(By.XPATH, "./div/div[1]/div/a/div[2]").text
            except NoSuchElementException:
                tmp_data_dict['requirement-list'] = 'N/A'  
            try:
                tmp_data_dict['CPprofile-list'] = ele.find_element(By.XPATH, "./div/div[1]/div/div/div/div[2]").text
            except NoSuchElementException:
                tmp_data_dict['CPprofile-list'] = 'N/A'  

            # # Wait for element to be visible and scroll to the visible area
            # driver.execute_script('arguments[0].scrollIntoView();', ele)
            # ele = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(ele))
            
            # Open job information detail page
            ele.click()
            driver.switch_to.window(driver.window_handles[-1])
            time.sleep(60)

            # Extract job information detail
            # Job description
            try:
                tmp_data_dict['job-detail'] = driver.find_element(By.XPATH, "//dd[@data-selector='job-intro-content']").text   # responsibilities and requirement-detail
            except NoSuchElementException:
                tmp_data_dict['job-detail'] = 'N/A'  
            # Profile-detail
            try:
                tmp_data_dict['enterprise-detail'] = driver.find_element(By.XPATH, "//div[@class='paragraph-box']//div[1]").text   # enterprise profile-detail
            except NoSuchElementException:
                tmp_data_dict['enterprise-detail'] = 'N/A'  
            tmp_data_list.append(tmp_data_dict)
            info_list.append(tmp_data_dict)

            # Return overview page
            driver.close()
            driver.switch_to.window(driver.window_handles[-1])
            # time.sleep(5)

        # Save current page data
        with open(f'tmp_data_dir/{page_number}.json', 'w') as ftd:
            json.dump(tmp_data_list, ftd, ensure_ascii=False)
        tdf = tmp_data_df.from_records(tmp_data_list)
        tdf.to_csv(f'tmp_data_dir/{page_number}.csv', index=False)

        # Go to next page
        next_button = driver.find_element(By.XPATH, "//li[@title='Next Page']//button[1]")
        if next_button.get_attribute('disabled') == 'true':
            break
        else:
            next_button.click()
            page_number += 1
            time.sleep(60)

    # Close the browser
    driver.quit()

# Capture exceptions and exit the browser
except Exception as e:
    print(e)
    driver.quit()


# Save information data
with open('job1.json', 'w') as f:
    json.dump(info_list, f, ensure_ascii=False)

df = info_df.from_records(info_list)
df.to_csv('job1.csv', index=False)

In [ ]:
import re

print(len(info_list))
# print(type(info_list[2]['job-detail'].encode('unicode_escape').decode('utf-8')))
# pprint(info_list[60:])

# s = info_list[60:]
# def find_incompatible_characters(string):
#     # 定义匹配非法字符的正则表达式
#     pattern = re.compile(r'[^\x09\x0A\x0D\x20-\uD7FF\uE000-\uFFFD\u10000-\u10FFFF]')

#     # 查找非法字符
#     incompatible_characters = re.findall(pattern, string)

#     return incompatible_characters

# # 示例用法
# string = "Hello, <world>!"
# incompatible_chars = find_incompatible_characters(s)
# if incompatible_chars:
#     print("字符串中存在与XML不兼容的字符：", incompatible_chars)
# else:
#     print("字符串中不存在与XML不兼容的字符")

# Implementation Based On Requests Lib

In [ ]:
# %%file liepin.py

# Code to scrape job information from the website goes here

import time
import requests
import json
import pandas as pd
from pprint import pprint
from bs4 import BeautifulSoup

def parse_page(page_sauce: bytes | str) -> list:
    recruitment_list = list ()
    soup = BeautifulSoup(page_sauce, 'html.parser')
    job_info_set = soup.select("body > div.m-content-container > div:nth-child(2) > ul > li")

    # Extract data
    for element in job_info_set:
        recruitment = dict()

        # Locate key node
        date_section_tag = element.find(name='section', attrs={'class': 'm-list-line_date'})
        content_section_tag = element.find(name='section', attrs={'class': 'm-list-line_content'})
        item_section_tag = element.find(name='section', attrs={'class': 'm-list-xyzp_item_top'})
        item_ul_tag = element.find(name='ul', attrs={'class': 'm-list-xyzp_item_bottom'})

        # Pre-build data structure
        date = list(date_section_tag.stripped_strings)
        date.reverse()
        date = ''.join(date)
        
        name = content_section_tag.h3.a.string.lstrip()

        time_hits_location = ['N/A', 'N/A', 'N/A',]
        time_hits_location[0] = list(item_section_tag.stripped_strings)[0].split(' ')[-1]  # 时间
        time_hits_location[1] = list(item_section_tag.stripped_strings)[1:][0].split('：')[1] # 点击量
        time_hits_location[2] = list(item_section_tag.stripped_strings)[1:][1].split('：')[1] # 地点
        
        orgname_orgnature_orgindustry = ['N/A', 'N/A', 'N/A',]  # 名称， 性质， 行业
        org_item_list = [org_item for org_item in item_ul_tag.children if org_item != '\n']
        for i, li in enumerate(org_item_list):
            orgname_orgnature_orgindustry[i] = li.h6.string

        # Build a dictionary to store info 
        try:
            recruitment['日期'] = date
        except:
            recruitment['日期'] = 'N/A'
        try:
            recruitment['招聘会名称'] = name
        except:
            recruitment['招聘会名称'] = 'N/A'
        recruitment['招聘时间'] = time_hits_location[0]
        recruitment['点击量'] = time_hits_location[1]
        recruitment['招聘地点'] = time_hits_location[2]
        recruitment['单位名称'] = orgname_orgnature_orgindustry[0]
        recruitment['单位性质'] = orgname_orgnature_orgindustry[1]
        recruitment['单位行业'] = orgname_orgnature_orgindustry[2]
        recruitment_list.append(recruitment)
        time.sleep(1)
    return recruitment_list

def main(path, json_file_name, csv_file_name):
    job_list = []
    # Prepera request
    page_no = 1
    while True:
        url = f""
        user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.62'
        # accept = 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
        # accept_encoding = 'gzip, deflate, br'
        # accept_language = 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6,ja;q=0.5'
        # referer = ''
        # cookie = 'Hm_lvt_f3a1f3ff8d120912a12ab2de0a25b16a=1693614414; pageSize=12; jdjy.session.id=1eb587a0acb045cd95956b6d37a86c1f; pageNo=63; Hm_lpvt_f3a1f3ff8d120912a12ab2de0a25b16a=1693642813'

        headers = {"User-Agent": user_agent,
                # "Accept": accept,
                # "Accept-Encoding": accept_encoding,
                # "Accept-Language": accept_language,
                # "Referer": referer,
                # "Cookie": cookie
                }
        

        response = requests.get(url, headers=headers, )
        soup = BeautifulSoup(response.text, 'html.parser')

        job_list += parse_page(response.text)
        
        # Go to next page 
        # print(soup)
        state = soup.find(attrs={'class': 'page-link'}, string='下一页 »').has_attr('onclick')

        if not state:
            break
        else:
            page_no += 1
            # print(page_no)
        

    json_file = ''
    with open(path, 'w') as f:
        json.dump(job_list, f, ensure_ascii=False)

    job_df = pd.DataFrame.from_records(job_list)
    job_df.to_csv(path, index=False)
    pprint(job_list)
    # # Save page source
    # with open('zhaopin.html', 'wb') as fr:
    #     fr.write(response.content)
main()